In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
model_name = "Llama-2-70b-chat-hf"
model_path = f"/gpfs/user/jsh/{model_name}"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.float16).cuda(7)

f"{torch.cuda.memory_reserved() / 1024 ** 3} GiB"

In [ ]:
prompt = ["What is deep learning?"]
inputs = tokenizer(prompt, return_tensors="pt").input_ids.cuda(7)
inputs

In [ ]:
for max_token in [1024, 2048, 4096]:
    outputs = model.generate(
        inputs,
        do_sample=True,
        top_k=50,
        top_p=0.92,
        # min_length=10,
        # max_length=50,
        temperature=0.9,
        repetition_penalty=1.5,
        no_repeat_ngram_size=3,
        max_new_tokens=max_token,
    )
    sentences = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    print(f"[{i}] {torch.cuda.memory_reserved() / 1024 ** 3} GiB")
    torch.cuda.empty_cache()